In [ ]:
import tensorflow as tf
import numpy as np
import seaborn as sns
import mlflow
import dagshub
import json
import matplotlib.pyplot as plt
from dagshub import dagshub_logger
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, classification_report
from kerastuner.tuners import RandomSearch

In [ ]:
# MLflow - Dagshub initialization
mlflow.set_tracking_uri("https://dagshub.com/alfoCaiazza/FingerSpellIT.mlflow")

dagshub.init(repo_owner='alfoCaiazza', repo_name='FingerSpellIT', mlflow=True)
dagshub_log = dagshub_logger(metrics_path="metrics", hparams_path="params")

In [ ]:
# To avoid OOM errors, setting GPU Memory Consuption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print(f"GPU: {gpu}")
    tf.config.experimental.set_memory_growth(gpu, True) # Keeping the use of memory limited to prevent errors

In [ ]:
train_dir = '../data/processed/train'
val_dir = '../data/processed/val'
test_dir = '../data/processed/test'

# Automatically creates a dataset form the referred directory. Load the full dataset, shuffle = True ensures randomness
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=128,
    shuffle=True,
    seed=123
)

class_names = train_ds.class_names

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(224, 224),
    batch_size=128,
    shuffle=True,
    seed=123
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=128,
    shuffle=True,
    seed=123
)

In [ ]:
# Pre-processing sequential model
preprocessing_model = tf.keras.Sequential([
    # Normalizing the images for ResNet50 model (0,1)
    tf.keras.layers.Rescaling(1./255), 

    # # Data augmentation:
    # tf.keras.layers.RandomFlip("horizontal"),  # Simulating left hand
    # tf.keras.layers.RandomRotation(0.1),       # Rotations, max ±10%

    # # Gaussiano noise
    # tf.keras.layers.Lambda(
    #     lambda x: tf.clip_by_value(
    #         x + tf.random.normal(tf.shape(x), mean=0.0, stddev=0.05), 0.0, 1.0
    #     )
    # ),

    # tf.keras.layers.RandomZoom(0.1)
])

train_ds = train_ds.map(lambda x, y: (preprocessing_model(x), y))
val_ds = val_ds.map(lambda x, y: (preprocessing_model(x), y))
test_ds = test_ds.map(lambda x, y: (preprocessing_model(x), y))

In [ ]:
# Importing net for transfer learning
def build_model(hp):
    base_model = ResNet50(
        weights='imagenet',
        include_top=False, # Excludes the first layer
        input_shape=(224,224, 3) # Specifing input shape
    )

    # Freezing net layers
    base_model.trainable = False

    # Adding more layer to the net from its output and adapting it to a multi-class classification task 
    x = GlobalAveragePooling2D()(base_model.output)
    activation = hp.Choice('activation', ['relu', 'sigmoid', 'tanh'])
    x = Dense(hp.Int('units', 128, 512, step=128), activation=activation)(x)
    predictions = Dense(22, activation='softmax')(x) 

    # Creiamo il modello finale
    model = Model(inputs=base_model.input, outputs=predictions)

    # Compiliamo il modello
    model.compile(
        optimizer=Adam(hp.Choice('lr', [1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-5)
]

In [ ]:
with mlflow.start_run(run_name="Model parameters"):
    epochs = 50
    mlflow.log_param('epochs', epochs)

    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=10,
        executions_per_trial=1,
        directory='tuner_logs',
        project_name='FingerSpellIT optimization'
    )

    tuner.search(train_ds, validation_data=val_ds, epochs=5, callbacks=callbacks)

    best_model = tuner.get_best_models(num_models=1)[0]
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]

    for param in best_hps.values:
        mlflow.log_param(param, best_hps.get(param))

    # Log metriche finali del trial
    for metric_name, value in best_trial.metrics.metrics.items():
        # metric_name potrebbe essere 'val_accuracy', ecc.
        if 'value' in value:
            mlflow.log_metric(metric_name, value['value'])

    # Salving the model
    model_path = "../model/model_fingerspelling.h5"
    best_model.save(model_path)
    mlflow.log_artifact(model_path)



In [ ]:
# Converte in liste ordinate per plotting
def extract_metric(metric_history):
    return [step["value"] for step in metric_history]

# Estrai l'elenco degli step e valori di metriche
train_loss = best_trial.metrics.get_history('loss')
val_loss = best_trial.metrics.get_history('val_loss')
train_acc = best_trial.metrics.get_history('accuracy')
val_acc = best_trial.metrics.get_history('val_accuracy')

loss_values = extract_metric(train_loss)
val_loss_values = extract_metric(val_loss)
acc_values = extract_metric(train_acc)
val_acc_values = extract_metric(val_acc)

# Plot
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(loss_values, label='Training Loss')
plt.plot(val_loss_values, label='Validation Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.subplot(1, 2, 2)
plt.plot(acc_values, label='Training Accuracy')
plt.plot(val_acc_values, label='Validation Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.tight_layout()
plt.show()

In [ ]:
test_loss, test_acc = best_model.evaluate(test_ds)
print(f"Evaluation accuracy: {test_acc:.4f}")

y_true = np.concatenate([y for x, y in test_ds])
y_pred = np.argmax(best_model.predict(test_ds), axis=1)

In [ ]:
report_path = '../model/artifacts/classification_report.json'
report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
with open(report_path, "w") as f:
    json.dump(report, f, indent=4)

mlflow.log_artifact(report_path)
print(report)

In [ ]:
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d',  xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Real')
plt.show()

In [ ]:
plt.tight_layout()
confusion_path = "../model/artifacts/confusion_matrix.png"
plt.savefig(confusion_path)
plt.close()

mlflow.log_artifact(confusion_path)

In [ ]:
mlflow.end_run()